# Solar Farm Applications in Nature Reserves(SSSI)

In [1]:
import leafmap
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, overlay
from shapely.geometry import Point
import matplotlib.pyplot as plt

from datetime import date
from dateutil.relativedelta import relativedelta




In [2]:
!python --version

Python 3.8.12


### Set the date range for the API call:


In [3]:

today = date.today()
today = today.strftime('%Y-%m-%d')
three_months_ago = date.today() + relativedelta(months=-3)
three_months_ago = three_months_ago.strftime('%Y-%m-%d')

### Get dataframe from API

In [4]:
# https://www.planit.org.uk/api/

df=pd.read_json('https://www.planit.org.uk/api/applics/json?start_date='+ three_months_ago +'&'+ 'end_date=' + today +'&search=solar&compress=on')


# extract new dataframe from dictionary of records

solar = pd.json_normalize(df['records'])
solar.head()

,name,uid,scraper_name,description,address,postcode,url,associated_id,app_size,app_state,...,other_fields.meeting_date,other_fields.agent_tel,other_fields.appeal_result,other_fields.decision_date,other_fields.decision_issued_date,other_fields.development_type,other_fields.first_advertised_date,other_fields.permission_expires_date,other_fields.applicant_company,other_fields.decision_published_date
0,Ashfield/V/2021/0745,V/2021/0745,Ashfield,Solar panel installation car park 1,"Sutton Community Academy, Sutton Centre, High ...",NG17 1EE,https://planning.ashfield.gov.uk/planning-appl...,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DorsetCouncil/P/HOU/2021/03594,P/HOU/2021/03594,DorsetCouncil,Change of Use and erection of a household grou...,Higher Barton Cottage Compton Road Over Compto...,DT9 4QY,https://planning.dorsetcouncil.gov.uk/plandisp...,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ashfield/V/2021/0746,V/2021/0746,Ashfield,Solar panels installation car park 2,"Sutton Community Academy, Sutton Centre, High ...",NG17 1EE,https://planning.ashfield.gov.uk/planning-appl...,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NorthNorfolk/IS3/21/2743,IS3/21/2743,NorthNorfolk,Installation of solar panels on south facing p...,Friday Cottage 7 Friday Market Place Walsingha...,NR22 6DB,https://idoxpa.north-norfolk.gov.uk/online-app...,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ashfield/V/2021/0744,V/2021/0744,Ashfield,Imstallation of solar panels - car park 3,"Sutton Community Academy, Sutton Centre, High ...",NG17 1EE,https://planning.ashfield.gov.uk/planning-appl...,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
solar.shape
#solar.to_csv('solar.csv')

(1020, 83)

### Filtering the data

In [6]:
solar['description'] = solar['description'].str.lower() 

In [7]:
# exclude (!consider risks)

my_exclusions = ['residential', 'roof']
pattern = '|'.join(my_exclusions)

solar_excluded = solar.loc[~solar['description'].str.contains(pattern)]
# solar = solar[(~solar['description'].str.contains('roof'))] #residential
# solar.shape
solar_excluded.shape

(521, 83)

In [8]:
mylist = ['watts', 'mega', 'solar park', 'land '] # land is poor discriminator!
pattern = '|'.join(mylist)

solar_parks = solar_excluded.loc[solar_excluded['description'].str.contains(pattern)]

In [9]:
# include without the excludes

# mylist = ['watts', 'mega', 'solar park', 'land '] # land is poor discriminator!
# pattern = '|'.join(mylist)

# solar_parks = solar.loc[solar['description'].str.contains(pattern)]

In [10]:
solar_parks.shape

(39, 83)

In [11]:
pd.set_option('max_colwidth', 400)

solar_parks.description.head(10)

6                                                                                                                                                                                        screening opinion under the environment impact assessment regulations 2017 - installation of two new supporting towers along the andover-amesbury circuit providing a connection to solar farm on land at hatherden farm
11                                           works to facilitate the repair, restoration and alterations of existing barns to provide recreational facilities and erection of a free standing pool building linked to the barns, installation of ground source heat pumps and solar panel array, change of use of land to domestic curtilage, alterations to access and installation of a package treatment plant
14     works to facilitate the repair, restoration and alterations of existing barns to provide recreational facilities and erection of a free standing pool building linked to the barns, installat

In [29]:
# caution: choose which filtered list to use for analysis:
#following analysis uses 'solar' so set your filtered list to that

solar = solar_parks 



### Issues with missing spatial columns

In [30]:
# get me the entries without any spatial data
missing_spatials = ['postcode', 'location_x', 'location_y', 'other_fields.lat', 'other_fields.lng', 'other_fields.easting', 'other_fields.northing']


solar_invisible = solar[solar[missing_spatials].isna().all(1)]
print(solar_invisible.shape)
#solar_invisible.shape

(9, 83)


In [31]:
# make a csv of cases with hyperlinks for their geography

#header = ['postcode', 'description', 'location_x', 'location_y', 'other_fields.lat', 'other_fields.lng', 'other_fields.easting', 'other_fields.northing']
solar_invisible.to_csv('solar_invisible.csv') #columns = header

### Fix geometries for cases that can be fixed quickly

In [33]:
# choosing column with max chance of simple fix for geography
solar['location.coordinates'].iloc[8]

[-1.878687, 51.63884]

In [34]:
# split out location coordinates into two cols

# handle missing values (i.e. possible cases will be missing)
dropped = solar['location.coordinates'].dropna()
solar[['X', 'Y']] = pd.DataFrame(dropped.tolist(), index=dropped.index)

/home/tim/.local/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [36]:
solar.shape

(39, 85)

In [37]:
# drop cases where geography is missing

solar = solar.dropna(subset = ['X', 'Y'])

In [38]:
solar.shape

(30, 85)

### Get the Nature reserve mapping SSSIs

In [39]:
# attribution: UK Department of Environment Food & Rural Affairs
# https://environment.data.gov.uk/DefraDataDownload/?mapService=NE/SitesOfSpecialScientificInterestEngland&Mode=spatial

SSSIs = gpd.read_file("data/Sites_of_Special_Scientific_Interest_England.shp")
#SSSIs.set_index(['geometry'])

In [40]:
SSSIs.head(1)


,sssi_name,sssi_area,easting,northing,latitude,longitude,reference,status,gid,ensisid,gis_file,area,easting0,northing0,gis_date,version,st_area_sh,st_perimet,geometry
0,Allen Confluence Gravels,4.777198,379993.104106,558767.08019,54:55:23N,2:19:10W,NY799587,Notified,1003435.0,1005624.0,None,4.777198,379993.104106,558767.08019,20031218,1.0,47771.979838,1957.148648,"POLYGON ((380021.211 558598.082, 380011.202 558601.101, 380001.868 558605.190, 379985.798 558610.799, 379973.497 558613.098, 379956.199 558614.198, 379952.299 558614.098, 379936.699 558612.298, 379914.899 558609.199, 379896.702 558609.199, 379888.202 558609.699, 379878.901 558610.699, 379829.398 558618.876, 379813.304 558614.897, 379784.899 558610.479, 379769.596 558608.299, 379763.899 558608...."


In [41]:
# SSSIs.dtypes

In [42]:
# SSSIs.crs

In [43]:
# SSSIs.total_bounds

### Working towards plot of all applications and likely solar park applications in SSSIs

In [44]:


print('Solar applications: complete list =', solar.shape[0], ', filtered list =', solar_parks.shape[0])

Solar applications: complete list = 30 , filtered list = 39


In [45]:
solarGDF = gpd.GeoDataFrame(solar, geometry=gpd.points_from_xy(solar['X'], solar['Y']))  # crs={'init':'epsg:27700'}
solarGDF.shape

(30, 86)

In [46]:
# solarGDF.total_bounds

In [47]:
solarGDF = solarGDF.set_crs(epsg=4326, inplace=True)


In [48]:
# solarGDF.total_bounds

In [49]:
solar27700 = solarGDF.to_crs(epsg=27700)
solar27700.total_bounds

array([183555.28559353,  54075.98224614, 646483.41794618, 669916.45790296])

In [50]:
# solarGDF['geometry'] = solarGDF.geometry.buffer(50000)

In [51]:
# solarGDF.head(1)

In [52]:
# solar27700.shape

In [53]:
# fig, ax = plt.subplots(figsize=(20, 16))

# SSSIs.plot(ax = ax, edgecolor='black',  )

# solar27700.plot(ax=ax, marker='o', color='red', markersize=2)

# plt.show()

In [54]:
pointinpolys = gpd.sjoin(SSSIs, solar27700, predicate='contains', how='inner' ) # 

In [55]:
pointinpolys.shape

(1, 105)

In [56]:
polyswithpoints = gpd.sjoin(solar27700, SSSIs, predicate='within', how='inner')

In [57]:
polyswithpoints.shape

(1, 105)

In [58]:
m = leafmap.Map(center=[50.5,-4], zoom=8, height="700px", width="700px")
m.add_gdf(pointinpolys, layer_name="SSSIs with solar planning applications")
m.add_gdf(polyswithpoints, layer_name="solar planning applications")
m

# this map currently plots any application for 'solar' that lands inside an SSSI

Map(center=[50.5, -4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…


### Try to buffer the points for solar applications just ouside nature reserves - NOT WORKING YET:

In [59]:
solar27700.head(1)

,name,uid,scraper_name,description,address,postcode,url,associated_id,app_size,app_state,...,other_fields.decision_date,other_fields.decision_issued_date,other_fields.development_type,other_fields.first_advertised_date,other_fields.permission_expires_date,other_fields.applicant_company,other_fields.decision_published_date,X,Y,geometry
6,TestValley/21/03016/SCRN,21/03016/SCRN,TestValley,screening opinion under the environment impact assessment regulations 2017 - installation of two new supporting towers along the andover-amesbury circuit providing a connection to solar farm on land at hatherden farm,Land To The West Of Chalkcroft Lane Penton Mewsey Andover Hampshire SP11 0HT,SP11 0HT,https://view-applications.testvalley.gov.uk/online-applications/applicationDetails.do?activeTab=summary&keyVal=R0YSEDQC0I000,None,Medium,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.528374,51.23602,POINT (433022.248 148643.474)


In [60]:
# set a buffer, e.g. 500 = 500m radius, so up to 1km away

solar27700['geometry'] = solar27700.geometry.buffer(500)

In [61]:
bufferswithpolys = gpd.sjoin(SSSIs, solar27700, predicate='intersects', how='inner' ) # 

In [65]:
bufferswithpolys.shape

(3, 105)

In [62]:
polyswithbuffers = gpd.sjoin(solar27700, SSSIs, predicate='intersects', how='inner')

In [66]:
polyswithbuffers.shape

(3, 105)

In [63]:
m = leafmap.Map(center=[50.5,-4], zoom=8, height="700px", width="700px")
style = {
    "stroke": True,
    "color": "#0000ff",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.1,
}
m.add_gdf(polyswithbuffers, layer_name="SSSIs with solar planning applications", style=style)
m.add_gdf(bufferswithpolys, layer_name="solar planning applications")
m

Map(center=[50.5, -4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
# in theory export map as html but layers are omitted. Fixable?
m.to_html('solarSSSImap.html')

In [ ]:


polybuffered = overlay(SSSIs, bufferpoint, how='intersection')

In [ ]:
polybuffered.shape

In [ ]:
pointbuffered = gpd.sjoin(SSSIs, buffer1km, op='contains', how='inner' )

In [ ]:
pointbuffered.shape

In [ ]:
m = leafmap.Map(center=[54,-2], zoom=6, height="800px", width="450px")
m.add_gdf(SSSIs, layer_name="SSSIs")
m

In [ ]:
cols = solar.columns

In [ ]:
SSSI_example = SSSIs.loc[SSSIs['sssi_name'] == 'Taw-Torridge Estuary']
# SSSI_example.crs
# SSSI_example.head()

In [ ]:
fake_solar = pd.read_csv('fake_applications.csv')

In [ ]:
fake_solar.head()

In [ ]:
fakeGDF = gpd.GeoDataFrame(fake_solar, geometry=gpd.points_from_xy(fake_solar['X'], fake_solar['Y']))  # crs={'init':'epsg:27700'}
fakeGDF.shape

In [ ]:
fakeGDF = fakeGDF.set_crs(epsg=4326, inplace=True)


In [ ]:
fakeGDF.crs == SSSI_example.crs

In [ ]:
fakeGDF.total_bounds

In [ ]:
SSSI_example.total_bounds

In [ ]:
fake27700 = fakeGDF.to_crs(epsg=27700)

In [ ]:
SSSI27700 = SSSI_example.to_crs(epsg=27700)

In [ ]:
SSSI27700.geometry

In [ ]:
fake27700.geometry

In [ ]:
fake27700.total_bounds

In [ ]:
SSSI27700.total_bounds

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))

#polyswithpoints.plot(ax = ax, edgecolor='black',  )

fake27700.plot(ax=ax, marker='o', color='red', markersize=2)
SSSI27700.plot(ax=ax)

plt.show()